In [1]:
import sklearn
import numpy
import json
import hashlib
import operator

from typing import List, Optional, Dict

# Experiment: Use Statistical Method detect "Bad Structure"

In [2]:
with open("./states.json", "r") as f: all_data = json.load(f)

## Definition of "Bad Structure"

If a structure exists in more than 10 consecutive states, we define such structure as a "bad structure" as it can't be resolved for a long time and will occupy position in 2048 board.

Below defines some utility functions

In [3]:
def state_and(s1, s2):
    # For simplicity, suppose both s1 and s2 are rectangular array
    assert len(s1) == len(s2) and len(s1[0]) == len(s2[0])
    # Create an empty 2D array with same shape as s1
    result = [[0] * len(s1[0]) for _ in range(len(s1))]
    for row in range(len(s1)):
        for col in range(len(s1[0])):
            if s1[row][col] == s2[row][col]: result[row][col] = s1[row][col]
    return result

def detect_bad_struct(s_list: List[List[List]]):
    s_0 = s_list[0]
    for s in s_list: s_0 = state_and(s_0, s)
    return s_0

def is_valid_bad_struct(badStruct):
    cond1 = sum([sum(row) for row in badStruct]) > 32
    flag = False
    for row in range(len(badStruct) - 1):
        for col in range(len(badStruct[0]) - 1):
            if badStruct[row][col] != 0 and badStruct[row + 1][col] != 0 and \
                badStruct[row][col + 1] != 0 and badStruct[row + 1][col + 1] != 0:
                flag = True
    return cond1 and flag

def combine_dicts(dict1, dict2):
    for key in dict2:
        if key in dict1: dict1[key] = dict2[key] + dict1[key]
        else: dict1[key] = dict2[key]
    return dict1

def extract_bad_structs(sList: List[List[List]], sequenceL: int=10):
    badStructs = dict()
    for i in range(0, len(sList) - sequenceL):
        badStruct = detect_bad_struct(sList[i:i + sequenceL])
        if is_valid_bad_struct(badStruct):
            badStructs[str(badStruct)] = 1
    return badStructs


## Detect Bad Structures in the Recorded State

Use `detect_bad_struct`, we can extract bad structure for all recorded state sequence.

In [4]:
bad_struct_count = dict()
for i, k in enumerate(list(all_data.keys())[:]):
    bad_struct_count = combine_dicts(bad_struct_count, extract_bad_structs(all_data[k], sequenceL=20))

In [5]:
bad_struct_list = [(bad_struct_count[k], k) for k in bad_struct_count]
bad_struct_list.sort(reverse=True)
bad_structs = [k for k in bad_struct_count]
with open("./bad_structs.json", "w") as f:
    json.dump(bad_structs, f)